# 介绍langchain的回调模块和用法

## 简单同步回调，对于LLMChain来说，和verbose作用相同

In [2]:
import os

from langchain.callbacks import StdOutCallbackHandler

from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()

llm = OpenAI(
    temperature=0,
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    base_url = os.getenv("OPENAI_BASE_URL"),
 
)

prompt = PromptTemplate.from_template("{year}年是中国生肖年的哪年？")

chain_callback =  LLMChain(
    llm=llm,
    prompt=prompt,
    # callbacks = [handler]
    verbose = True
)

chain_callback.invoke({"year":1987})



> Entering new LLMChain chain...
Prompt after formatting:
1987年是中国生肖年的哪年？

> Finished chain.


{'year': 1987, 'text': '\n\n1987年是中国生肖年的兔年。'}

## 使用 FileCallbackHandler 进行日志输出

In [3]:
from loguru import logger

from langchain.callbacks import FileCallbackHandler

logfile = 'output.log'

logger.add(logfile,colorize=True,enqueue = True)

handler = FileCallbackHandler(logfile)

chain =  LLMChain(
    llm=llm,
    prompt=prompt,
    callbacks = [handler],
    verbose = True
)

response = chain.invoke({"year":1988})
logger.info(response)



> Entering new LLMChain chain...
Prompt after formatting:
1988年是中国生肖年的哪年？


2024-01-31 16:03:40.488 | INFO     | __main__:<module>:19 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}



> Finished chain.


In [4]:
with open('output.log','r',encoding='utf-8') as file:
    content = file.read()
    
print(content)



> Entering new LLMChain chain...
Prompt after formatting:
1988年是中国生肖年的哪年？

> Finished chain.
2024-01-30 10:14:16.758 | INFO     | __main__:<module>:16 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}
2024-01-30 10:14:16.758 | INFO     | __main__:<module>:16 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}


> Entering new LLMChain chain...
Prompt after formatting:
1988年是中国生肖年的哪年？

> Finished chain.
2024-01-30 10:46:51.005 | INFO     | __main__:<module>:14 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}
2024-01-30 10:46:51.005 | INFO     | __main__:<module>:14 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}
2024-01-30 10:46:51.005 | INFO     | __main__:<module>:14 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}


> Entering new LLMChain chain...
Prompt after formatting:
1988年是中国生肖年的哪年？

> Finished chain.
2024-01-31 16:03:40.488 | INFO     | __main__:<module>:19 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}



## 使用get_openai_callback回调实现token计数

In [5]:
from langchain.callbacks import get_openai_callback


llm = OpenAI(
    temperature=0,
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    base_url = os.getenv("OPENAI_BASE_URL"),
)

with get_openai_callback() as cb:
    llm.invoke("牛顿什么时候发现的万有引力定律？")

total_tokens = cb.total_tokens
assert total_tokens >0

print(total_tokens)

46


In [6]:
with get_openai_callback() as cb:
    logfile = 'output.log'
    logger.add(logfile,colorize=True,enqueue = True)
    handler = FileCallbackHandler(logfile)

    chain =  LLMChain(
    llm=llm,
    prompt=prompt,
    callbacks = [handler],
    verbose = True
    )

    response = chain.invoke({"year":1988})
    logger.info(response)

total_tokens = cb.total_tokens
assert total_tokens >0

print(total_tokens)    



> Entering new LLMChain chain...
Prompt after formatting:
1988年是中国生肖年的哪年？


2024-01-31 16:08:02.635 | INFO     | __main__:<module>:14 - {'year': 1988, 'text': '\n\n1988年是中国生肖年的龙年。'}



> Finished chain.
29
